# Training, Validation and Test

In [39]:
import pandas as pd

df = pd.read_csv("../data/classifier/concat.csv")
df

,id,text,label,num_words,clean_text
0,1576050330114363392,@joicehasselmann Bolsonaro NÃO quer aproveitad...,-1,7,bolsonaro não quer aproveitadores como você.
1,1576050330110160901,o bolsonaro é um anjo enviado de deus para nos...,1,22,o bolsonaro é um anjo enviado de deus para nos...
2,1576050318852640769,@AndradeRNegro2 @Loucosdovalle Bolsonaro é con...,1,28,bolsonaro é contra o aborto e defende a vida...
3,1576050318596399104,@Victorrmsk É a milésima vez que ele fala ao v...,0,15,é a milésima vez que ele fala ao vivo que não...
4,1576050309822287873,"@OGloboPolitica Bolsonaro é tão fake, q até o ...",-1,18,"bolsonaro é tão fake, q até o laranja q ele l..."
...,...,...,...,...,...
697,1597340353430171648,Hahaha e os patriotas na chuva \nhttps://t.co/...,-1,7,hahaha e os patriotas na chuva \n
698,1597340352733904897,@PATRlOTAS Esse idiota @BolsonaroSP não se ele...,-1,34,"esse idiota não se elege nunca mais, o brasi..."
699,1597340328813506560,@rafaelgloves Depois disso se o pai dele nao d...,-1,28,depois disso se o pai dele nao der o contra g...
700,1597340303308259329,@ducavendish Modinha de cretinos. Bolsonaro é ...,1,51,modinha de cretinos. bolsonaro é o único resp...


In [40]:
import numpy as np

inputs = df['clean_text'].to_numpy()
target = df['label'].to_numpy()

In [41]:
from sentence_transformers import SentenceTransformer

model_transform = SentenceTransformer('../distiluse-base-multilingual-cased-v1')
model_transform

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [42]:
embeddings = model_transform.encode(inputs)

In [105]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(embeddings, target, test_size=0.2, stratify = target, random_state=42)

In [106]:
print("Length train:", len(y_train))
print("Length test:", len(y_test))

Length train: 561
Length validation: 141


In [137]:
from imblearn.over_sampling import SMOTE 

#Balancing dataset using SMOTE

sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)

# KNN

In [138]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)

param_grid = {'n_neighbors' : list(range(2, 40)), 'weights' : ['distance'], 'metric': ['cosine']}

clf = GridSearchCV(estimator = KNeighborsClassifier(n_jobs = -1), param_grid = param_grid, cv = skf, n_jobs=-1)
clf.fit(x_train, y_train)
df_results = pd.DataFrame(clf.cv_results_)

In [139]:
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003006,0.000000e+00,0.024597,0.000799,cosine,2,distance,"{'metric': 'cosine', 'n_neighbors': 2, 'weight...",0.600000,0.641379,0.682759,0.652778,0.736111,0.662605,0.045334,2
1,0.003005,3.814697e-07,0.027996,0.008124,cosine,3,distance,"{'metric': 'cosine', 'n_neighbors': 3, 'weight...",0.634483,0.675862,0.675862,0.638889,0.701389,0.665297,0.025190,1
2,0.002002,8.970816e-04,0.024800,0.001599,cosine,4,distance,"{'metric': 'cosine', 'n_neighbors': 4, 'weight...",0.620690,0.655172,0.668966,0.625000,0.694444,0.652854,0.027587,3
3,0.001801,3.998770e-04,0.030400,0.003007,cosine,5,distance,"{'metric': 'cosine', 'n_neighbors': 5, 'weight...",0.627586,0.655172,0.627586,0.625000,0.673611,0.641791,0.019376,4
4,0.001400,4.912327e-04,0.038999,0.006232,cosine,6,distance,"{'metric': 'cosine', 'n_neighbors': 6, 'weight...",0.606897,0.634483,0.613793,0.618056,0.666667,0.627979,0.021369,5
5,0.001401,4.895712e-04,0.031199,0.002562,cosine,7,distance,"{'metric': 'cosine', 'n_neighbors': 7, 'weight...",0.606897,0.620690,0.586207,0.631944,0.659722,0.621092,0.024612,7
6,0.001400,4.889896e-04,0.037199,0.007250,cosine,8,distance,"{'metric': 'cosine', 'n_neighbors': 8, 'weight...",0.606897,0.620690,0.606897,0.638889,0.659722,0.626619,0.020295,6
7,0.001401,4.896892e-04,0.037198,0.007386,cosine,9,distance,"{'metric': 'cosine', 'n_neighbors': 9, 'weight...",0.593103,0.600000,0.579310,0.645833,0.652778,0.614205,0.029506,15
8,0.002001,3.368374e-06,0.034998,0.004691,cosine,10,distance,"{'metric': 'cosine', 'n_neighbors': 10, 'weigh...",0.593103,0.606897,0.579310,0.638889,0.666667,0.616973,0.031748,12
9,0.001399,4.904283e-04,0.039713,0.007307,cosine,11,distance,"{'metric': 'cosine', 'n_neighbors': 11, 'weigh...",0.593103,0.620690,0.558621,0.652778,0.659722,0.616983,0.037678,11


In [140]:
model_knn = clf.best_estimator_
model_knn

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=3,
                     weights='distance')

In [141]:
y_pred_knn = model_knn.predict(x_test)
y_pred_knn

array([-1, -1,  0, -1,  1, -1,  1,  0,  1,  0,  1,  0, -1, -1,  0,  0,  0,
        1,  1, -1,  0, -1,  1, -1,  1, -1,  0,  0, -1, -1, -1,  1,  1,  1,
       -1,  0,  1,  0, -1, -1,  1,  1,  0,  1, -1,  1,  0,  0,  0,  0,  1,
       -1, -1, -1, -1, -1,  0,  1,  1,  1,  1,  0, -1, -1, -1,  1,  0, -1,
        1,  1,  0,  1, -1,  1,  1,  1,  0, -1, -1,  0, -1,  0,  0,  1,  1,
        1,  1, -1,  0,  1, -1,  0, -1,  1, -1,  1, -1, -1,  1,  0,  0, -1,
       -1,  1,  0,  1,  1, -1, -1,  0,  0, -1,  0, -1,  0,  1,  1,  0, -1,
        1, -1,  1, -1,  1,  0,  1,  1, -1,  1,  0, -1, -1, -1,  1,  0,  0,
       -1, -1,  1, -1,  1], dtype=int64)

In [142]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

          -1       0.58      0.49      0.53        61
           0       0.64      0.42      0.51        59
           1       0.34      0.81      0.48        21

    accuracy                           0.51       141
   macro avg       0.52      0.58      0.51       141
weighted avg       0.57      0.51      0.51       141



# AdaBoostClassifier

In [143]:
from sklearn.ensemble import AdaBoostClassifier

param_grid = {'n_estimators' : [100, 500, 1000, 1500, 2000], 'learning_rate' : [0.5, 1.0, 1.5]}

model_ada = AdaBoostClassifier(random_state = 42)

clf = GridSearchCV(estimator = model_ada, param_grid = param_grid, cv = skf, n_jobs=-1)
clf.fit(x_train, y_train)
df_results = pd.DataFrame(clf.cv_results_)


In [144]:
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.408412,0.144706,0.022400,0.000800,0.5,100,"{'learning_rate': 0.5, 'n_estimators': 100}",0.558621,0.751724,0.758621,0.722222,0.777778,0.713793,0.079614,13
1,17.330121,0.385001,0.115710,0.006001,0.5,500,"{'learning_rate': 0.5, 'n_estimators': 500}",0.600000,0.724138,0.758621,0.791667,0.756944,0.726274,0.066653,12
2,34.710168,0.611589,0.220015,0.010684,0.5,1000,"{'learning_rate': 0.5, 'n_estimators': 1000}",0.620690,0.786207,0.793103,0.784722,0.770833,0.751111,0.065610,9
3,50.504662,0.763099,0.324525,0.014044,0.5,1500,"{'learning_rate': 0.5, 'n_estimators': 1500}",0.655172,0.758621,0.779310,0.784722,0.777778,0.751121,0.048778,8
4,67.340258,0.489986,0.477927,0.082420,0.5,2000,"{'learning_rate': 0.5, 'n_estimators': 2000}",0.641379,0.751724,0.779310,0.770833,0.798611,0.748372,0.055578,10
5,3.304055,0.029064,0.022001,0.000634,1.0,100,"{'learning_rate': 1.0, 'n_estimators': 100}",0.579310,0.696552,0.703448,0.673611,0.743056,0.679195,0.054728,14
6,16.401333,0.167171,0.108400,0.001743,1.0,500,"{'learning_rate': 1.0, 'n_estimators': 500}",0.682759,0.696552,0.813793,0.736111,0.791667,0.744176,0.051395,11
7,33.855957,0.443232,0.219907,0.006858,1.0,1000,"{'learning_rate': 1.0, 'n_estimators': 1000}",0.696552,0.758621,0.793103,0.756944,0.812500,0.763544,0.039563,5
8,51.690997,0.825816,0.353512,0.025937,1.0,1500,"{'learning_rate': 1.0, 'n_estimators': 1500}",0.668966,0.765517,0.786207,0.770833,0.840278,0.766360,0.055433,4
9,69.048398,0.834470,0.457548,0.016262,1.0,2000,"{'learning_rate': 1.0, 'n_estimators': 2000}",0.662069,0.751724,0.800000,0.756944,0.833333,0.760814,0.057694,6


In [145]:
model_ada = clf.best_estimator_
model_ada

AdaBoostClassifier(learning_rate=1.5, n_estimators=1500, random_state=42)

In [146]:
y_pred_ada = model_ada.predict(x_test)
y_pred_ada

array([ 0, -1,  0, -1,  0,  0,  0, -1, -1,  0,  1, -1, -1, -1,  0,  0,  0,
        1,  1, -1,  0, -1, -1, -1,  0, -1, -1,  0, -1, -1, -1,  0,  0,  0,
       -1,  0,  0,  0, -1, -1,  1,  0, -1,  0,  0,  1,  0, -1,  0, -1,  0,
       -1, -1, -1,  0, -1,  0,  0,  0,  1,  0,  0, -1, -1,  0,  1,  0, -1,
        0,  1,  0, -1,  0,  0,  1,  0,  0, -1, -1,  0, -1,  0,  0,  0, -1,
       -1, -1, -1,  0,  0, -1,  0,  0,  0,  0,  1, -1, -1, -1,  0, -1, -1,
        0, -1,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0,  0, -1,
        0, -1,  1, -1, -1,  0,  0,  0, -1, -1,  0, -1, -1, -1,  1, -1, -1,
       -1, -1,  0,  0,  0], dtype=int64)

In [147]:
print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

          -1       0.74      0.74      0.74        61
           0       0.66      0.76      0.71        59
           1       0.83      0.48      0.61        21

    accuracy                           0.71       141
   macro avg       0.74      0.66      0.68       141
weighted avg       0.72      0.71      0.71       141



# Test custom data

In [148]:
def classifier(model, text):
    embeddings_test = model_transform.encode(text)
    y_pred = model.predict(embeddings_test)
    df_results = pd.DataFrame({'Text' : text, 'label' : y_pred})

    return df_results

In [149]:
x = ['Bolsonaro é o melhor presidente possível', 'Lula sempre deu o seu melhor pelo país', 'Ciro coitado, não serve pra nada', 'Todos os candidatos são ruins', 'Gostei muito dessa proposta', 'Bolsonaro foi ao congresso', 'Essa eleição é uma piada mesmo', 'Simplesmente horrível política', 'Não consigo entender como tem gente que não gosta de política no Brasil', 'Estou muito contente com o meu canditato!', 'Não acho é nada sobre isso', 'Sério que você acha o lula bom? kkkkkkkkkkkkk', "Ele é um ladrão velho", "Eleições são uma perda de tempo mesmo", 'Qualquer candidato eleito vai continuar a mesma coisa', '5 dias pro Lula ganhar no primeiro turno em galera', 'O brasil já está desacreditado de política nos dias atuais', 'insuportável essa época de política', 'Vai deixar de gostar do Neymar por conta de posição política? Eu nunca gostei do Neymar!.']

# KNN

In [150]:
df_results = classifier(model_knn, x)

In [151]:
from IPython.display import HTML

HTML(df_results.to_html())

,Text,label
0,Bolsonaro é o melhor presidente possível,1
1,Lula sempre deu o seu melhor pelo país,-1
2,"Ciro coitado, não serve pra nada",1
3,Todos os candidatos são ruins,-1
4,Gostei muito dessa proposta,1
5,Bolsonaro foi ao congresso,1
6,Essa eleição é uma piada mesmo,1
7,Simplesmente horrível política,-1
8,Não consigo entender como tem gente que não gosta de política no Brasil,1
9,Estou muito contente com o meu canditato!,1


# AdaBoostClassifier

In [153]:
df_results = classifier(model_ada, x)

In [154]:
from IPython.display import HTML

HTML(df_results.to_html())

,Text,label
0,Bolsonaro é o melhor presidente possível,1
1,Lula sempre deu o seu melhor pelo país,1
2,"Ciro coitado, não serve pra nada",0
3,Todos os candidatos são ruins,-1
4,Gostei muito dessa proposta,1
5,Bolsonaro foi ao congresso,0
6,Essa eleição é uma piada mesmo,0
7,Simplesmente horrível política,-1
8,Não consigo entender como tem gente que não gosta de política no Brasil,-1
9,Estou muito contente com o meu canditato!,1
